In [1]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

In [2]:
torch.manual_seed(42)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cpu


In [4]:
df = pd.read_csv('fashion-mnist_test.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.shape

(10000, 785)

In [6]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [7]:
type(X)

numpy.ndarray

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_train = X_train/255.0
X_test = X_test/255.0

In [10]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]


In [11]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [12]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [15]:
import optuna

In [16]:
class MyNN(nn.Module):

  def __init__(self, num_features):
    super().__init__()

    self.model = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, 64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(64, 10)
    )

  def forward(self, x):
    return self.model(x)

In [17]:
learning_rate = 0.1
epochs = 100

In [18]:
model = MyNN(X_train.shape[1])
model = model.to(device)

loss_function = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

In [ ]:
for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    y_pred = model(batch_features)

    loss = loss_function(y_pred, batch_labels)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1}, Loss: {avg_loss}')

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch: 1, Loss: 0.8581342548131943
Epoch: 2, Loss: 0.6335705033540726
Epoch: 3, Loss: 0.5604625252485276
Epoch: 4, Loss: 0.5121229358315468
Epoch: 5, Loss: 0.4866853140592575
Epoch: 6, Loss: 0.46904879933595656
Epoch: 7, Loss: 0.43828579485416413
Epoch: 8, Loss: 0.43570101249217985
Epoch: 9, Loss: 0.42033442556858064
Epoch: 10, Loss: 0.407668751090765
Epoch: 11, Loss: 0.39728713178634645
Epoch: 12, Loss: 0.3952499574124813
Epoch: 13, Loss: 0.37465396827459335
Epoch: 14, Loss: 0.3710413386821747
Epoch: 15, Loss: 0.35220867800712585
Epoch: 16, Loss: 0.3531287810206413
Epoch: 17, Loss: 0.3525577348172665
Epoch: 18, Loss: 0.3305539863705635
Epoch: 19, Loss: 0.32618040540814397
Epoch: 20, Loss: 0.31762652775645256
Epoch: 21, Loss: 0.31823271775245665
Epoch: 22, Loss: 0.30898830041289327
Epoch: 23, Loss: 0.30335209277272224
Epoch: 24, Loss: 0.2942466750741005
Epoch: 25, Loss: 0.2877521752566099
Epoch: 26, Loss: 0.29728505958616736
Epoch: 27, Loss: 0.28994928930699826
Epoch: 28, Loss: 0.28776

In [ ]:
model.eval()

In [ ]:
correct = 0
total = 0

with torch.no_grad():
  for batch_features, batch_labels in test_loader:

    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    y_pred = model(batch_features)
    _, predicted = torch.max(y_pred, 1)

    total += batch_labels.size(0)
    correct += (predicted == batch_labels).sum().item()

  accuracy = correct/total * 100
  print(f'Accuracy: {accuracy}')

In [ ]:
correct = 0
total = 0

with torch.no_grad():
  for batch_features, batch_labels in train_loader:

    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    y_pred = model(batch_features)
    _, predicted = torch.max(y_pred, 1)

    total += batch_labels.size(0)
    correct += (predicted == batch_labels).sum().item()

  accuracy = correct/total * 100
  print(f'Accuracy: {accuracy}')